# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 59 new papers today
          21 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/20 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.09909


extracting tarball to tmp_2209.09909...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.09914


extracting tarball to tmp_2209.09914...

...

/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.10386/natbib.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2209.10395... done.
Retrieving document from  https://arxiv.org/e-print/2209.10408


extracting tarball to tmp_2209.10408...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.10455


extracting tarball to tmp_2209.10455... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.09914-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.09914) | **CLusters in the Uv as EngineS (CLUES): I. Survey presentation \& FUV  spectral analysis of the stellar light**  |
|| M. Sirressi, et al. -- incl., <mark>S. Hernandez</mark>, <mark>L. J. Smith</mark>, <mark>E. Schinnerer</mark> |
|*Appeared on*| *2022-09-22*|
|*Comments*| *Accepted for publication on the Astronomical Journal on 16th September 2022. 26 pages, 12 figures*|
|**Abstract**| The CLusters in the Uv as EngineS (CLUES) survey is a Cosmic Origins Spectrograph (COS) campaign aimed at acquiring the 1130 to 1770 {\AA}, restframe spectroscopy of very young (<20 Myr) and massive (>10^4 solar masses) star clusters in galaxies that are part of the Hubble treasury program Legacy ExtraGalactic Uv Survey (LEGUS). In this first paper of a series, we describe the CLUES sample consisting of 20 young star clusters and report their physical properties as derived by both multi-wavelength photometry and far-UV (FUV) spectroscopy with Hubble Space Telescope (HST). Thanks to the synergy of the two different datasets we build a coherent picture of the diverse stellar populations found in each region (with sizes of 40 to 160 pc). We associate the FUV-brightest stellar population to the central targeted star cluster and the other modeled population to the diffuse stars that are included in the COS aperture. We observe better agreement between photometric and spectroscopic ages for star clusters younger than 5 Myr. For clusters older than 5 Myr, photometry and spectroscopy measurements deviate, with the latter producing older ages, due to the degeneracy of photometric models. FUV spectroscopy enables us to better constrain the stellar metallicities, a parameter that optical colors are insensitive to. Finally, the derived E(B-V) are quite similar, with a tendency for FUV spectroscopy to favor solutions with higher extinctions. The recovered masses are in agreement within a factor of 2 for all the clusters. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.09944-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.09944) | **Deep learning reconstruction of sunspot vector magnetic fields for  forecasting solar storms**  |
|| Dattaraj B. Dhuri, Shamik Bhattacharjee, Shravan M. Hanasoge, <mark>Sashi Kiran Mahapatra</mark> |
|*Appeared on*| *2022-09-22*|
|*Comments*| *19 Pages, 11 Figures, Accepted for publication in ApJ*|
|**Abstract**| Solar magnetic activity produces extreme solar flares and coronal mass ejections, which pose grave threats to electronic infrastructure and can significantly disrupt economic activity. It is therefore important to appreciate the triggers of explosive solar activity and develop reliable space-weather forecasting. Photospheric vector-magnetic-field data capture sunspot magnetic-field complexity and can therefore improve the quality of space-weather prediction. However, state-of-the-art vector-field observations are consistently only available from Solar Dynamics Observatory/Helioseismic and Magnetic Imager (SDO/HMI) since 2010, with most other current and past missions and observational facilities such as Global Oscillations Network Group (GONG) only recording line-of-sight (LOS) fields. Here, using an inception-based convolutional neural network, we reconstruct HMI sunspot vector-field features from LOS magnetograms of HMI as well as GONG with high fidelity (~ 90% correlation) and sustained flare-forecasting accuracy. We rebuild vector-field features during the 2003 Halloween storms, for which only LOS-field observations are available, and the CNN-estimated electric-current-helicity accurately captures the observed rotation of the associated sunspot prior to the extreme flares, showing a striking increase. Our study thus paves the way for reconstructing three solar cycles worth of vector-field data from past LOS measurements, which are of great utility in improving space-weather forecasting models and gaining new insights about solar activity. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.09909-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.09909) | **Pulse Timing Discovery of a Three-Day Companion to the Hot Subdwarf BPM  36430**  |
|| <mark>Bryce A. Smith</mark>, et al. -- incl., <mark>J.J. Hermes</mark> |
|*Appeared on*| *2022-09-22*|
|*Comments*| *7 pages, 5 figures, 4 tables. Accepted for publication in the Astrophysical Journal*|
|**Abstract**| Hot subdwarf B stars are core-helium burning objects that have undergone envelope stripping, likely by a binary companion. Using high-speed photometry from the Transiting Exoplanet Survey Satellite, we have discovered the hot subdwarf BPM 36430 is a hybrid sdBV_rs pulsator exhibiting several low-amplitude g-modes and a strong p-mode pulsation. The latter shows a clear, periodic variation in its pulse arrival times. Fits to this phase oscillation imply BPM 36430 orbits a barycenter approximately 10 light-seconds away once every 3.1 d. Using the CHIRON echelle spectrograph on the CTIO 1.5-m telescope, we confirm the reflex motion by detecting a radial velocity variation with semi-amplitude, period, and phase in agreement with the pulse timings. We conclude that a white dwarf companion with minimum mass of 0.42 Msun orbits BPM 36430. Our study represents only the second time a companion orbiting a pulsating hot subdwarf or white dwarf has been detected from pulse timings and confirmed with radial velocities. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.10408-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.10408) | **Wavelet analysis of the transient QPOs in MAXI J1535$-$571 with  Insight-HXMT**  |
|| X. Chen, et al. -- incl., <mark>P. Zhang</mark>, <mark>Q. Liu</mark>, <mark>S. Zhang</mark>, <mark>S. N. Zhang</mark> |
|*Appeared on*| *2022-09-22*|
|*Comments*| *10 pages, MNRAS in press*|
|**Abstract**| Using wavelet analysis and power density spectrum, we investigate two transient quasi-periodic oscillations (QPOs) observed in MAXI J1535$-$571 observed with Insight-HXMT. The transient QPOs have a centroid frequency of $\sim 10$ Hz with a FWHM $\sim 0.6$ Hz and an rms amplitude $\sim 14\%$. Energy spectra of QPO and non-QPO regimes are also separated and analyzed, and the spectra become softer with higher $E_{cut}$ in the non-QPO regime compared to the QPO regime. Our results suggest that the transient QPOs detected in MJD 58016 and 58017 are still the type-C QPO, and the source remains in its HIMS. The duration of all type-C QPO signals based on wavelet is positively correlated with the mean count rate above $\sim 10$ keV, implying appearance of QPOs in different time scales should be coupled with the corona. The transient QPO properties could be related to the jet or flares, perhaps the partial ejection of the corona is responsible for the disappearance of the type-C QPO. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.10455-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.10455) | **High energy Millihertz quasi-periodic oscillations in 1A 0535+262 with  Insight-HXMT challenge current models**  |
|| Ruican Ma, et al. -- incl., <mark>Lian Tao</mark>, <mark>Shuang-Nan Zhang</mark>, <mark>Liang Zhang</mark>, <mark>Liming Song</mark>, <mark>Shu Zhang</mark>, <mark>Hexin Liu</mark>, <mark>Lingda Kong</mark>, <mark>Panping Li</mark> |
|*Appeared on*| *2022-09-22*|
|*Comments*| *13 pages, 7 figures. Accepted for publication in MNRAS*|
|**Abstract**| We studied the millihertz quasi-periodic oscillation (mHz QPO) in the 2020 outburst of the Be/X-ray binary 1A 0535+262 using Insight-HXMT data over a broad energy band. The mHz QPO is detected in the 27-120 keV energy band. The QPO centroid frequency is correlated with the source flux, and evolves in the 35-95 mHz range during the outburst. The QPO is most significant in the 50-65 keV band, with a significance of ~ 8 sigma, but is hardly detectable (<2 sigma) in the lowest (1-27 keV) and highest (>120 keV) energy bands. Notably, the detection of mHz QPO above 80 keV is the highest energy at which mHz QPOs have been detected so far. The fractional rms of the mHz QPO first increases and then decreases with energy, reaching the maximum amplitude at 50-65 keV. In addition, at the peak of the outburst, the mHz QPO shows a double-peak structure, with the difference between the two peaks being constant at ~0.02 Hz, twice the spin frequency of the neutron star in this system. We discuss different scenarios explaining the generation of the mHz QPO, including the beat frequency model, the Keplerian frequency model, the model of two jets in opposite directions, and the precession of the neutron star, but find that none of them can explain the origin of the QPO well. We conclude that the variability of non-thermal radiation may account for the mHz QPO, but further theoretical studies are needed to reveal the physical mechanism. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.09927-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.09927) | **Mixing Interstellar Clouds Surrounding the Sun**  |
|| Paweł Swaczyna, et al. -- incl., <mark>Jeffrey L. Linsky</mark> |
|*Appeared on*| *2022-09-22*|
|*Comments*| *13 pages, 5 figures, 1 table, accepted for publication in ApJL*|
|**Abstract**| On its journey through the Galaxy, the Sun passes through diverse regions of the interstellar medium. High-resolution spectroscopic measurements of interstellar absorption lines in spectra of nearby stars show absorption components from more than a dozen warm partially ionized clouds within 15 pc of the Sun. The two nearest clouds - the Local Interstellar Cloud (LIC) and Galactic (G) cloud - move toward each other. Their bulk heliocentric velocities can be compared with the interstellar neutral helium flow velocity obtained from space-based experiments. We combine recent results from Ulysses, IBEX, and STEREO observations to find a more accurate estimate of the velocity and temperature of the very local interstellar medium. We find that, contrary to the widespread viewpoint that the Sun resides inside the LIC, the locally observed velocity of the interstellar neutral helium is consistent with a linear combination of the velocities of the LIC and G cloud, but not with either of these two velocities. This finding shows that the Sun travels through a mixed-cloud interstellar medium composed of material from both these clouds. Interactions between these clouds explain the substantially higher density of the interstellar hydrogen near the Sun and toward stars located within the interaction region of these two clouds. The observed asymmetry of the interstellar helium distribution function also supports this interaction. The structure and equilibrium in this region require further studies using in situ and telescopic observations. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.10347-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.10347) | **Presolar Silicon Carbide Grains of Types Y and Z: Their Strontium and  Barium Isotopic Compositions and Stellar Origins**  |
|| <mark>Nan Liu</mark>, et al. |
|*Appeared on*| *2022-09-22*|
|*Comments*| *Accepted by EPJA*|
|**Abstract**| We report the Sr and Ba isotopic compositions of 18 presolar SiC grains of types Y (11) and Z (7), rare types commonly argued to have formed in lower-than-solar metallicity asymptotic giant branch (AGB) stars. We find that the Y and Z grains show higher 88Sr/87Sr and more variable 138Ba/136Ba ratios than mainstream (MS) grains. According to FRANEC Torino AGB models, the Si, Sr, and Ba isotopic compositions of our Y and Z grains can be consistently explained if the grains came from low mass AGB stars with 0.15 Zsun <= Z < 1.00 Zsun, in which the 13C neutron exposure for the slow neutron-capture process is greatly reduced with respect to that required by MS grains for a 1.0 Z8 AGB star. This scenario is in line with the previous finding based on Ti isotopes, but it fails to explain the indistinguishable Mo isotopic compositions of MS, Y, and Z grains. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.10161-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.10161) | **The PAU Survey & Euclid: Improving broad-band photometric redshifts with  multi-task learning**  |
|| L. Cabayol, et al. -- incl., <mark>E. Franceschi</mark> |
|*Appeared on*| *2022-09-22*|
|*Comments*| *20 pages, 16 figures*|
|**Abstract**| Current and future imaging surveys require photometric redshifts (photo-z) to be estimated for millions of galaxies. Improving the photo-z quality is a major challenge to advance our understanding of cosmology. In this paper, we explore how the synergies between narrow-band photometric data and large imaging surveys can be exploited to improve broad-band photometric redshifts. We use a multi-task learning (MTL) network to improve broad-band photo-z estimates by simultaneously predicting the broad-band photo-z and the narrow-band photometry from the broad-band photometry. The narrow-band photometry is only required in the training field, which enables better photo-z predictions also for the galaxies without narrow-band photometry in the wide field. This technique is tested with data from the Physics of the Accelerating Universe Survey (PAUS) in the COSMOS field. We find that the method predicts photo-z that are 14% more precise down to magnitude i_AB<23, while reducing the outlier rate by 40% with respect to the baseline network mapping broad-band colours to only photo-zs. Furthermore, MTL significantly reduces the photo-z bias for high-redshift galaxies, improving the redshift distributions for tomographic bins with z>1. Applying this technique to deeper samples is crucial for future surveys like \Euclid or LSST. For simulated data, training on a sample with i_AB <23, the method reduces the photo-z scatter by 15% for all galaxies with 24<i_AB<25. We also study the effects of extending the training sample with photometric galaxies using PAUS high-precision photo-zs, which further reduces the photo-z scatter. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error sequence item 1: expected str instance, TexNode found</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.09914.md
    + _build/html/tmp_2209.09914/./figures/ages.png
    + _build/html/tmp_2209.09914/./figures/metals.png
    + _build/html/tmp_2209.09914/./figures/ebvs.png
    + _build/html/tmp_2209.09914/./figures/masses.png
    + _build/html/tmp_2209.09914/./figures/color_plot_CLUESsel_pad_MW.png
    + _build/html/tmp_2209.09914/./figures/age_mass_CLUESsel_pad_MW.png
    + _build/html/tmp_2209.09914/./figures/M74-2-good.png
    + _build/html/tmp_2209.09914/./figures/M74-2-good_ciii.png
    + _build/html/tmp_2209.09914/./figures/M74-2-good_nv.png
    + _build/html/tmp_2209.09914/./figures/M74-2-good_civ.png
    + _build/html/tmp_2209.09914/./figures/NGC1566-2-bad.png
    + _build/html/tmp_2209.09914/./figures/NGC1566-2-bad_ciii.png
    + _build/html/tmp_2209.09914/./figures/NGC1566-2-bad_nv.png
    + _build/html/tmp_2209.09914/./figures/NGC1566-2-bad_civ.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\Rsun}{\hbox{R_\odot}}$
$\newcommand{\tr}{\hbox{t_{\rm relax}}}$
$\newcommand{\mv}{\hbox{M_{\rm V}}}$
$\newcommand{\lsun}{\hbox{L_\odot}}$
$\newcommand{\msun}{\hbox{M_\odot}}$
$\newcommand{\mstar}{\hbox{M_\star}}$
$\newcommand{\t4}{\hbox{t_{\rm 4}}}$
$\newcommand{\fmid}{\hbox{F_{\rm MID}}}$
$\newcommand{\reff}{\hbox{R_{\rm eff}}}$
$\newcommand{\dndt}{\hbox{\frac{d{\rm N}}{dt}}}$
$\newcommand{\cbound}{\hbox{C_{\rm bound}}}$
$\newcommand{\relation}{\hbox{SFR vs. M_{V}^{\rm brightest}}}$
$\newcommand{\clrelation}{\hbox{CFR vs. M_{V}^{\rm brightest}}}$
$\newcommand{\mvten}{\hbox{M_{\rm V}^{10{\rm Myr}}}}$
$\newcommand{\lir}{\hbox{L_{\rm IR}}}$
$\newcommand{\msunyr}{\hbox{M_\odotyr^{-1}}}$
$\newcommand{\dsfr}{\hbox{M_\odotyr^{-1}kpc^{-2}}}$
$\newcommand{\Ne}{\hbox{N_{\rm{e}}}}$
$\newcommand{\Te}{\hbox{T_{\rm{e}}}}$
$\newcommand{\kms}{\hbox{km s^{-1}}}$
$\newcommand{\cm3}{\hbox{cm^{-3}}}$
$\newcommand{\one}{ {\sc i}}$
$\newcommand{\two}{ {\sc ii}}$
$\newcommand{\three}{ {\sc iii}}$
$\newcommand{\four}{ {\sc iv}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{$\t$exttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{$\t$extrm{#1}}$
$\newcommand{$\hl$}[1]{$\t$extrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\Rsun$}{\hbox{R_\odot}}$
$\newcommand{$\tr$}{\hbox{t_{\rm relax}}}$
$\newcommand{$\mv$}{\hbox{M_{\rm V}}}$
$\newcommand{$\lsun$}{\hbox{L_\odot}}$
$\newcommand{$\msun$}{\hbox{M_\odot}}$
$\newcommand{$\mstar$}{\hbox{M_\star}}$
$\newcommand{$\t$4}{\hbox{t_{\rm 4}}}$
$\newcommand{$\fmid$}{\hbox{F_{\rm MID}}}$
$\newcommand{$\reff$}{\hbox{R_{\rm eff}}}$
$\newcommand{$\dndt$}{\hbox{\frac{d{\rm N}}{dt}}}$
$\newcommand{$\cbound$}{\hbox{C_{\rm bound}}}$
$\newcommand{$\relation$}{\hbox{SFR vs. M_{V}^{\rm brightest}}}$
$\newcommand{$\clrelation$}{\hbox{CFR vs. M_{V}^{\rm brightest}}}$
$\newcommand{$\mv$ten}{\hbox{M_{\rm V}^{10{\rm Myr}}}}$
$\newcommand{$\lir$}{\hbox{L_{\rm IR}}}$
$\newcommand{$\msun$yr}{\hbox{M_\odotyr^{-1}}}$
$\newcommand{$\dsfr$}{\hbox{M_\odotyr^{-1}kpc^{-2}}}$
$\newcommand{$\Ne$}{\hbox{N_{\rm{e}}}}$
$\newcommand{$\Te$}{\hbox{T_{\rm{e}}}}$
$\newcommand{$\kms$}{\hbox{km s^{-1}}}$
$\newcommand{$\cm$3}{\hbox{cm^{-3}}}$
$\newcommand{$\one$}{ {\sc i}}$
$\newcommand{$\t$wo}{ {\sc ii}}$
$\newcommand{$\t$hree}{ {\sc iii}}$
$\newcommand{$\four$}{ {\sc iv}}$</div>



<div id="title">

#  FUV spectral analysis of the stellar light

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.09914-b31b1b.svg)](https://arxiv.org/abs/2209.09914)<mark>Appeared on: 2022-09-22</mark> - _Accepted for publication on the Astronomical Journal on 16th September 2022. 26 pages, 12 figures_

</div>
<div id="authors">

M. Sirressi, et al. -- incl., <mark><mark>S. Hernandez</mark></mark>, <mark><mark>L. J. Smith</mark></mark>, <mark><mark>E. Schinnerer</mark></mark>

</div>
<div id="abstract">

**Abstract:** The CLusters in the Uv as EngineS (CLUES) survey is a Cosmic Origins Spectrograph (COS) campaign aimed at acquiring the 1130 to 1770\AArestframe spectroscopy of very young ($<$20 Myr) and massive ($>10^4$\msun) star clusters in galaxies that are part of the Hubble treasury program Legacy ExtraGalactic Uv Survey (LEGUS).In this first paper of a series, we describe the CLUES sample consisting of 20 young star clusters and report their physical properties as derived by both multi-wavelength photometry and far-UV (FUV) spectroscopy with Hubble Space Telescope (HST). Thanks to the synergy of the two different datasets we build a coherent picture of the diverse stellar populations found in each region (with sizes of 40 to 160 pc). We associate the FUV-brightest stellar population to the central targeted star cluster and the other modeled population to the diffuse stars that are included in the COS aperture. We observe better agreement between photometric and spectroscopic ages for star clusters younger than 5 Myr. For clusters older than 5 Myr, photometry and spectroscopy measurements deviate, with the latter producing older ages, due to the degeneracy of photometric models. FUV spectroscopy enables us to better constrain the stellar metallicities, a parameter that optical colors are insensitive to. Finally, the derived$E(B-V)$are quite similar, with a tendency for FUV spectroscopy to favor solutions with higher extinctions. The recovered masses are in agreement within a factor of 2 for all the clusters.

</div>

<div id="div_fig1">

<img src="tmp_2209.09914/./figures/ages.png" alt="Fig7.1" width="25%"/><img src="tmp_2209.09914/./figures/metals.png" alt="Fig7.2" width="25%"/><img src="tmp_2209.09914/./figures/ebvs.png" alt="Fig7.3" width="25%"/><img src="tmp_2209.09914/./figures/masses.png" alt="Fig7.4" width="25%"/>

**Figure 7. -** Values of the best-fit parameters for ages, metallicities, reddening attenuations and masses of our different models. The grey dots represent the values derived from photometry. The blue and orange squares are the values for $\t$extit{Pop 1} and $\t$extit{Pop 2} respectively, of our double population model of the FUV spectroscopy, with their size proportional to their light flux contribution at 1270 Å. The green stars are the average values of our multiple population model. (*fig:parameters*)

</div>
<div id="div_fig2">

<img src="tmp_2209.09914/./figures/color_plot_CLUESsel_pad_MW.png" alt="Fig2.1" width="50%"/><img src="tmp_2209.09914/./figures/age_mass_CLUESsel_pad_MW.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Parameter space spanned by the clusters belonging to the CLUES sample (cyan stars). The clusters that pass the first selection (class 1 and 2 with m$_{F275W} <$ 18 mag, $E(B-V)\leq$ 0.3 mag, age $\leq$ 30 Myr) are shown in both panels as grey dots. The left panel shows the FUV color, F275W$-$F438W (or F435W) vs. the optical color, F555W (or F606W)$-$F814W, of the clusters. Yggdrasil evolutionary tracks spanning the metallicity range of the LEGUS sample are included and the main age steps outlined. The arrow shows in which direction clusters would move if corrected by an internal reddening corresponding to $E(B-V)=0.3$ mag. In the right panel, we show the age vs. mass diagram of the sample. The violet band shows the detection limit corresponding m$_{F275W}=$18 mag as function of the metallicity (Z$=0.004$ and 0.02) and distance range of the LEGUS galaxies. (*fig:selection*)

</div>
<div id="div_fig3">

<img src="tmp_2209.09914/./figures/M74-2-good.png" alt="Fig6.1" width="12%"/><img src="tmp_2209.09914/./figures/M74-2-good_ciii.png" alt="Fig6.2" width="12%"/><img src="tmp_2209.09914/./figures/M74-2-good_nv.png" alt="Fig6.3" width="12%"/><img src="tmp_2209.09914/./figures/M74-2-good_civ.png" alt="Fig6.4" width="12%"/><img src="tmp_2209.09914/./figures/NGC1566-2-bad.png" alt="Fig6.5" width="12%"/><img src="tmp_2209.09914/./figures/NGC1566-2-bad_ciii.png" alt="Fig6.6" width="12%"/><img src="tmp_2209.09914/./figures/NGC1566-2-bad_nv.png" alt="Fig6.7" width="12%"/><img src="tmp_2209.09914/./figures/NGC1566-2-bad_civ.png" alt="Fig6.8" width="12%"/>

**Figure 6. -** Two targets (M-74-YSC2 and NGC-1566-YSC2) as examples of the fits to the FUV spectra (black line) with the single/double-population model and the multi-population model (purple and green lines respectively). The two panels below each target show a zoom into the P-Cygni lines C III, N V and C IV. The example targets chosen for this figure are a good and a poor agreement for the stellar age between single/double pop. model and multi-pop. model. (*fig:example*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

40  publications in the last 7 days.
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers